# Mapping out our data

We have some data that is geo-aware. This means that we can place it on a map.

To run this notebook you will need `geopandas`, `contextily`, `mapclassify` in addition to the fairly standard `pandas` and `matplotlib.pyplot`. The resulting plots have been saved, and are available to view directly.

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx

Load our data:

In [ ]:
all_coords = pd.read_csv('s3://geotermaldata/S3FluidInclusionGasAnalysisData/COSO Field/COSO_wells_coord_gen.csv',
                         skipinitialspace=True,
                         delimiter=r'\s*,',  # this removes some annoying spaces before the comma.
                         engine='python',
                        )

In [ ]:
fname = './data/cleaned_types.csv'
df = pd.read_csv(fname)
df

There are some columns that contain no data in the coordinates DataFrame, so we will drop those.

In [ ]:
all_coords.dropna(axis='columns', inplace=True)
all_coords

In [ ]:
all_coords.columns

In [ ]:
gdf = gpd.GeoDataFrame(df)

Although we have created `gdf` as a GeoDataFrame it does not have a proper geometry column yet.

To create this column we need to have each point as a geometry. The easiest approach is to use `.apply` to combine the `Long83` and `Lat83` columns into a proper `Point`. First we need to add the information from `all_coords` to `gdf`, which we can do with the `merge` method:

In [ ]:
gdf = gdf.merge(all_coords, left_on='Well ID', right_on='WellNumber')
gdf

Notice that we have more columns than we started with, which are the ones from `all_coords`. We have also lost some rows, which are those from our fluid inclusion data which do not have a matching well in the coordinates DataFrame.

Now we can use `.apply` to make a valid shapely `Point` based on the `Long83` and `Lat83` columns. This will give us a `geometry` column which will get used when plotting.

In [ ]:
from shapely.geometry import Point

def make_point(row):
    #print(row)
    return Point(float(row['Long83']), float(row['Lat83']))

In [ ]:
gdf['geometry'] = gdf.apply(make_point, axis=1)
gdf = gdf.set_geometry('geometry', crs="EPSG:4326")
gdf

We can also make a GeoDataFrame of all of the wells in much the same way. These have no fluid inclusion data associated with them though.

In [ ]:
all_wells = gpd.GeoDataFrame(all_coords)  # turn the wells into a GeoDataFrame
all_wells['geometry'] = all_wells.apply(make_point, axis=1)
all_wells.set_crs(epsg=4326, inplace=True)
all_wells

## Plotting Things!

Our wells with fluid inclusions are shown in orange, with the remainder in blue.

In [ ]:
fig, ax = plt.subplots()
all_wells.plot(ax=ax)
gdf.plot(ax=ax)

Since we are plotting maps, we need to have a single value per point to plot. We can use the maximum depth as something useful for now.

In [ ]:
wells = gpd.GeoDataFrame()
for group in gdf.groupby('Well ID'):
    wells = wells.append(group[1].loc[group[1]['Depth (ft)'] == group[1]['Depth (ft)'].max()])
    
wells

Notice that this has some wells that are duplicated, because there are more than one in the `all_coords` DataFrame (one is active and one is inactive. We can either ignore these or drop them later. I am just ignoring this for now, because the data is the same.

In [ ]:
for idx, well in enumerate(wells['Well ID']):
    try:
        if wells.iloc[idx-1]['Well ID'] == well:
            print(f'{idx-1} and {idx} are both well {well}.')
    except IndexError:
        continue

Adding some basemaps to our plots.

In [ ]:
tiles = ctx.providers.Stamen.Terrain
image = ctx.providers.Esri.WorldImagery

fig, ax = plt.subplots(figsize=(12,12))
all_wells.plot(ax=ax, c='white', ec='k', markersize=40)
wells.plot(ax=ax, ec='k', markersize=50, column='WellType', legend=True)
ctx.add_basemap(ax=ax, source=tiles, crs=4326, attribution='')
ctx.add_basemap(ax=ax, source=image, crs=4326, alpha=0.6, attribution='')
ctx.add_attribution(ax=ax, text=f'{tiles.attribution}\n{image.attribution}')

# Add labels for our wells of interest.
for well, lat, lon in zip(wells['Well ID'], wells['Lat83'],  wells['Long83']):
    geom = (float(lon)+0.001, float(lat)-0.001)
    ax.annotate(well, geom)
    
plt.tight_layout()
plt.savefig('./img/all_wells.png', dpi=300)

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
wells.plot(ax=ax, ec='k', markersize=50, column='WellType', legend=True)

ctx.add_basemap(ax=ax, source=tiles, crs=4326, attribution='')
ctx.add_basemap(ax=ax, source=image, crs=4326, alpha=0.6, attribution='')
ctx.add_attribution(ax=ax, text=f'{tiles.attribution}\n{image.attribution}')

for well, lat, lon in zip(wells['Well ID'], wells['Lat83'],  wells['Long83']):
    geom = (float(lon)+0.001, float(lat)-0.001)
    ax.annotate(well, geom)
    
plt.tight_layout()
plt.savefig('./img/fi_wells.png', dpi=300)

### Plotting maximum depths

The depths are in feet, so I am converting them to metres.

In [ ]:
wells['Depth (ft)'].describe()

In [ ]:
wells['Depth_m'] = wells['Depth (ft)'] * 0.3048
wells['Depth_m']

Now we can plot them easily:

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
wells.plot(ax=ax, ec='k', markersize=50,
           column='Depth_m',
           scheme='percentiles',
           legend=True,
           cmap=plt.cm.plasma,
          )

ctx.add_basemap(ax=ax, source=tiles, crs=4326, attribution='')
ctx.add_basemap(ax=ax, source=image, crs=4326, alpha=0.6, attribution='')
ctx.add_attribution(ax=ax, text=f'{tiles.attribution}\n{image.attribution}')

for well, lat, lon in zip(wells['Well ID'], wells['Lat83'],  wells['Long83']):
    geom = (float(lon)+0.001, float(lat)-0.001)
    #print(geom)
    ax.annotate(well, geom)
    
plt.tight_layout()
plt.savefig('./img/fi_wells_depth.png', dpi=300)

In [ ]:
gdf.to_file('./data/fi_wells.gpkg', driver='GPKG')

In [ ]:
all_wells.to_file('./data/well_locations.gpkg', driver='GPKG')